# n-Photon Compton Scattering Process

In this file, we set up an n-photon Compton scattering process. A Compton scattering
process looks like $k^n e^- \to k e^-$.

You can download this file as a [jupyter notebook](compton.ipynb).

In [1]:
using QEDFeynmanDiagrams

We need some of the packages of the [QEDjl-project](https://github.com/QEDjl-project) for base
functionality and the `ScatteringProcess` type.

In [2]:
using QEDcore
using QEDprocesses

Let's decide how many photons our electron interacts with:

In [3]:
n = 4;

Now we setup the scattering process accordingly. We consider all spin/polarization
combinations of the particles except for the incoming photons, where the polarizations are synced using `QEDbase.SyncedPolarization`.
This emulates all synced photons having the same, but still indefinite, polarization, for example from a laser.

In [4]:
proc = ScatteringProcess(
    (Electron(), ntuple(_ -> Photon(), n)...),     # incoming particles
    (Electron(), Photon()),                        # outgoing particles
    (AllSpin(), ntuple(_ -> SyncedPol(1), n)...),  # incoming particle spin/pols
    (AllSpin(), AllPol()),                         # outgoing particle spin/pols
)

generic QED process
    incoming: electron (all spins), photon (synced polarization 1), photon (synced polarization 1), photon (synced polarization 1), photon (synced polarization 1)
    outgoing: electron (all spins), photon (all polarizations)


The `feynman_diagrams` function returns an iterator for all possible Feynman diagrams
for this scattering process. With its `length` overload, we can check how many diagrams
there are. For an n-photon Compton process with `n` incoming photons, this should be
$(n+1)!$.

In [5]:
length(feynman_diagrams(proc))

120

Next, we can generate the DAG representing the computation for our scattering process'
squared matrix element. This uses [`ComputableDAGs.jl`](https://github.com/ComputableDAGs/ComputableDAGs.jl).

In [6]:
dag = generate_DAG(proc)

Graph:
  Nodes: Total: 2262, QEDFeynmanDiagrams.ComputeTask_PropagatePairs: 152, QEDFeynmanDiagrams.ComputeTask_Propagator: 30, 
         QEDFeynmanDiagrams.ComputeTask_Triple: 480, ComputableDAGs.DataTask: 1153, QEDFeynmanDiagrams.ComputeTask_CollectTriples: 16, 
         QEDFeynmanDiagrams.ComputeTask_Pair: 264, QEDFeynmanDiagrams.ComputeTask_CollectPairs: 152, QEDFeynmanDiagrams.ComputeTask_BaseState: 14, 
         QEDFeynmanDiagrams.ComputeTask_SpinPolCumulation: 1
  Edges: 4185
  Total Compute Effort: 0.0
  Total Data Transfer: 0.0
  Total Compute Intensity: 0.0


In this graph output you can see the number of nodes necessary to compute.
Note that for larger processes, the number of total nodes can be *lower* than
the number of Feynman diagrams, even with the added complexity of considering
multiple spin and polarization combinations. This is the result of efficient
reuse of reappearing parts of Feynman diagrams.

To continue, we will need [`ComputableDAGs.jl`](https://github.com/ComputableDAGs/ComputableDAGs.jl). Since `ComputableDAGs.jl` uses
`RuntimeGeneratedFunction`s as the return type of `ComputableDAGs.get_compute_function`, we need
to initialize it in our current module.

In [7]:
using ComputableDAGs
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

With the DAG, the process, and `RuntimeGeneratedFunctions` initalized,
we can now generate the actual computable function:

In [8]:
func = get_compute_function(dag, proc, cpu_st(), @__MODULE__);

Now we need an input for the function, which is a `QEDcore.PhaseSpacePoint`.
For now, we generate random momenta for every particle. In the future, QEDevents
will be able to generate physical `PhaseSpacePoint`s.

In [9]:
psp = PhaseSpacePoint(
    proc,
    PerturbativeQED(),
    PhasespaceDefinition(SphericalCoordinateSystem(), ElectronRestFrame()),
    tuple((rand(SFourMomentum) for _ in 1:number_incoming_particles(proc))...),
    tuple((rand(SFourMomentum) for _ in 1:number_outgoing_particles(proc))...),
)

PhaseSpacePoint:
    process: generic QED process "ekkkk -> ek"
    model: perturbative QED
    phasespace definition: spherical coordinates in electron rest frame
    incoming particles:
     -> incoming electron: [0.3130027363633211, 0.8257697537613388, 0.19887283603268358, 0.526298584076336]
     -> incoming photon: [0.22990041972909425, 0.5158759759388896, 0.6768654303678353, 0.5740868959913966]
     -> incoming photon: [0.38573108544062296, 0.9292740476441036, 0.41592523277078985, 0.527612748706132]
     -> incoming photon: [0.8686084267691285, 0.3642842582830721, 0.4874306366510571, 0.3159338754636305]
     -> incoming photon: [0.8914624933582852, 0.5467597117793941, 0.026463106053576713, 0.16925351048307158]
    outgoing particles:
     -> outgoing electron: [0.8374124852328247, 0.7372281998039045, 0.07158135748911165, 0.6443685663900897]
     -> outgoing photon: [0.02066278075149508, 0.8797612817360894, 0.12073013825877488, 0.8203682655471711]


Finally, we can test that the function actually runs and computes something by
simply calling it on the `PhaseSpacePoint`:

In [10]:
func(psp)

0.4402557439994318

If we want, we can benchmark the execution speed too:

In [11]:
using BenchmarkTools
@benchmark func($psp)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  42.670 μs … 105.267 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     43.502 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   45.267 μs ±   4.678 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▆▄▃▂▂▄▄▃▁▁▁▁             ▂▂                                ▁
  █████████████████▇▇▇▇▆▆▆▆▆▇███▇▆▇▆▇▇▆▆▆▆▆▅▅▅▅▃▅▄▄▅▄▄▃▂▄▄▄▂▅▄ █
  42.7 μs       Histogram: log(frequency) by time      67.8 μs <

 Memory estimate: 4.88 KiB, allocs estimate: 46.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*